# Movie Recommendation Systems
This notebook was created to solve the Kaggle EDSA Movie Recommendation Challenge.

Reccomendation sysytems are an integral part to any online user based service platform. In short an alogrithim is created that will reccomened you items (eg: movies) based on your history with past items.

For this challenge we are tasked to create a movie recommendation system, There were two paths that we could take.
Content Based Filtering and Collaborative Filtering, We chose to try out both in which we will get further into this notebook

### Problem statement:
This is a machine learning model that will predict how a user will rate a movie they have not yet viewed based on their historical preferences.

<img src="https://imageproxy.themaven.net/https%3A%2F%2Fimages.saymedia-content.com%2F.image%2FMTY5ODE3MTkxNzY2NTY2MDg4%2Fnative-nerd-movie-list-of-45.jpg?" width=55%>

## Installations

In [ ]:
# !pip install comet_ml

In [ ]:
# from comet_ml import Experiment
# experiment = Experiment(api_Key = vyyN9zt9jvAs1X1LBmL5WYbn1, project_name = 'unsupervised_predict', workspace = 'keane257')

In [ ]:
!pip install surprise

# 1. Import  Libraries
we start by importing the various libraries we will need

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# General
import numpy as np
import pandas as pd
import scipy as sp # <-- The sister of Numpy, used in our code for numerical efficientcy. 
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

# Machine Learning

 # scipy
from scipy.spatial.distance import cdist
from scipy import stats
from scipy import sparse
from scipy.sparse import csr_matrix

 # sklearn
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics import mean_squared_error
from sklearn.metrics import silhouette_score
from sklearn import preprocessing

 # surprise
import surprise 
from surprise import SVD
from surprise.model_selection import cross_validate
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise import NormalPredictor
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline
from surprise import BaselineOnly
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering
from surprise.accuracy import rmse
from surprise.model_selection import train_test_split
from surprise import KNNBaseline,SlopeOne,CoClustering

# Entity featurization and similarity computation
from sklearn.metrics.pairwise import cosine_similarity 
from sklearn.feature_extraction.text import TfidfVectorizer

# Libraries used during sorting procedures.
import operator # <-- Convienient item retrieval during iteration 
import heapq # <-- Efficient sorting of large lists

# Imported for our sanity
import warnings
warnings.filterwarnings('ignore')
import math
import os

# 2. Converting the Data into DataFrames and exploring metrics of the data

In [ ]:
tags=pd.read_csv(r'/kaggle/input/edsa-recommender-system-predict/tags.csv')
Train=pd.read_csv(r'/kaggle/input/edsa-recommender-system-predict/train.csv')
genome_scores=pd.read_csv(r'/kaggle/input/edsa-recommender-system-predict/genome_scores.csv')
genome_tags=pd.read_csv(r'/kaggle/input/edsa-recommender-system-predict/genome_tags.csv')
Movie=pd.read_csv(r'/kaggle/input/edsa-recommender-system-predict/movies.csv')
imdb=pd.read_csv(r'/kaggle/input/edsa-recommender-system-predict/imdb_data.csv')
test=pd.read_csv(r'/kaggle/input/edsa-recommender-system-predict/test.csv')
links=pd.read_csv(r'/kaggle/input/edsa-recommender-system-predict/links.csv')
submission=pd.read_csv(r'/kaggle/input/edsa-recommender-system-predict/sample_submission.csv')

train = Train.copy()
movies = Movie.copy()

In [ ]:
tags

In [ ]:
# d={'train':Train.shape,'movies':movies.shape,'links':links.shape,
#    'idmb':imdb.shape,'genome_tags':genome_tags.shape,'genome_scores':genome_scores.shape}
# data =pd.DataFrame(data=d,index=['rows','columns'])
# data

In [ ]:
# Print the number of records and the total number of movies
print('The dataset contains: ', len(train), ' ratings of ', len(movies), ' movies.')

In [ ]:
train.describe()

# 3. Exploratory Data Analyses (EDA)

A visual summary of the dataframe to identify patterns and trends and communicate the insights of the data through visual representation
We first merge the train and movie dataframe to get a more detailed information of the movieid thats on train dataframe to find the title and genre and then we rename the merged dataframes as df

## Movie Director Analyses
Using imdb_data to analyse the directors and the influence they have on the rating ,budget aswell as analysis of active directors based on the movies they produce with user ratings


In [ ]:
df = pd.merge(train,movies,on='movieId')
df2=pd.merge(imdb,df)


exploring the new dataframe to get statical analyses , to see any null values,data type and the structure of the dataframe

In [ ]:
df2.describe()

In [ ]:
print(f'The SHape of DataFrame: {df2.shape}')
print('\nInfo about the DataFrame:')
print(df2.info())
print(f'\nMAx rating of movie: {df2.rating.max()}')
print(f'\nMin rating of movie: {df2.rating.min()}')

Now that we have merged,we start by creating a copy to remove duplicates to get more accurate results on the number of movies

In [ ]:
df3=df2.copy()
df3.drop_duplicates('movieId',inplace=True)
df3.director.value_counts().head(10)

These are the top 10 directors who produced the most movies,Director Luc Besson has the highest movies produced as compared to other directors and the director with the least movies is Peter Farrelly. now that we know the producers with the most movie we will check the ratings with regars to the movies and the users have rated their movies.

In [ ]:
rating_directors = df3.groupby('director').agg({'rating': 'mean', 'movieId': 'count'})
top_5_rating_directors = rating_directors.rename(columns={'movieId': 'Movies Count'}).sort_values('rating', ascending=False)[:5]
top_5_rating_directors


The director with the top movie rating is Stephen Zotnowski, even though he produced least amount of movies he still has the the highest user movie ratings

In [ ]:
ax = top_5_rating_directors['rating'].plot.bar();
ax.set_ylabel('IMDB Rating')
ax.set_title('Average IMDB Rating of Top 5 Directors by IMDB Rating', y=1.02)
ax.set_xticklabels(top_5_rating_directors.index, rotation = 45);


These are the top 5 directors with the highest ratings even though they did not produce a lot movies their movies produced are rated as 5.0 . which means more movies produced by this directors are enjoyed by the viewers

In [ ]:
most_active_directors = df2['director'].value_counts()[df2['director'].value_counts() >= 5].head(10)
most_active_directors

The director with the most movie rated by the users is Quentin Tarantino this implies that his movies have the highest viewers compared to other movies

In [ ]:
active_director_movies = df2[df2.director.isin(most_active_directors.index.tolist())]
active_director_movies.head()

In [ ]:
active_director_stats = active_director_movies.groupby('director')['budget', 'rating'].mean()
active_director_stats.sort_values(by='rating',ascending=False)

These are the top 10 directors who have movies with the highest number of ratings Jonathan Nolan has the highest rating. this concludes that his movies are being watched and enjoyed by the users

## Ratings Analyses
**In this section we will first analyse the movies,users and their rating trends based on average votes and movies watched.**

In [ ]:
def human(num, units = 'M'):
    units = units.lower()
    num = float(num)
    if units == 'k':
        return str(num/10**3) + " K"
    elif units == 'm':
        return str(num/10**6) + " M"
    elif units == 'b':
        return str(num/10**9) +  " B"

In [ ]:
fig, ax = plt.subplots()
plt.title('Distribution of ratings over Training dataset', fontsize=15)
sns.countplot(df2.rating)
ax.set_yticklabels([human(item, 'M') for item in ax.get_yticks()])
ax.set_ylabel('No. of Ratings(Millions)')

plt.show()

From this bar graph we can see that very view movies are rated as 0.5 and most average rated movies are 4.0 with ratings over 2.5 million. now that we have seen the highest ratings we will evaluate the title of movies with the highest ratings

In [ ]:
highest_ratings = pd.DataFrame(df2.groupby('title')['rating'].mean().sort_values(ascending=False).head(5))
highest_ratings

Final Recourse is rated as the top movie with the highest rating, now that we have seen the ratings we need to find out how many users have voted for these movies

In [ ]:
highest_ratings=pd.DataFrame(df2.groupby('title')['rating'].count().sort_values(ascending=False)).head()
highest_ratings.reset_index()

Movie Shawshank Redemption has the highest number of ratings this means that this movie has been watched a lot by the viewers how ever even though its on the top ten most rated it does not have the highest rating .we will analyse this further by evaluating the number of ratings.

In [ ]:
print("Total data ")
print("-"*50)
print("\nTotal no of ratings :",Train.shape[0])
print("Total No of Users   :", len(np.unique(Train.userId)))
print("Total No of movies  :", len(np.unique(Train.movieId)))

From above we have 1000038 number of ratings with 162541 users and 48213 number of movies which means that movie Shawshank Redemption has 33.8% of the ratings this could be its the most watched movies hence it has the highest number of ratings. next step we look at the distibution of number of ratings

In [ ]:
highest_num_rate= pd.DataFrame(df.groupby('title')['rating'].mean())
highest_num_rate['num of ratings'] = pd.DataFrame(df.groupby('title')['rating'].count().sort_values(ascending=False))
plt.figure(figsize=(10,4))
highest_num_rate['num of ratings'].hist(bins=100);

In [ ]:
plt.figure(figsize=(10,4))
highest_num_rate['rating'].hist(bins=100);

In [ ]:
sns.jointplot(x='rating',y='num of ratings',data=highest_num_rate,alpha=0.5);

## Date and Time Analyses

In [ ]:
movies2=pd.merge(Movie,Train,on='movieId')

In [ ]:
def clean1(text):
    text =text.replace('(','')
    text =text.replace('|','')
    text =text.replace(')','')
    text =text.replace('-','')
    return text

In [ ]:
movies2.head()

In [ ]:
movies1=movies2.copy()
movies1['year'] = movies1['title'].str.split('(').str[1]
movies1['year']=movies1['year'].str.replace(")","")
movies1['year'] = pd.to_numeric(movies1.year, errors='coerce').fillna(0, downcast='infer')
movies1

In [ ]:
movies1 =movies1.replace(0, np.nan)
movies1 =movies1.replace(6, np.nan)
movies1 = movies1.dropna(how='all', axis=0)
movies1.dropna(inplace=True)
movies1.isnull().sum()
movies1['year'].astype(int)
movies1['year']=pd.to_datetime(movies1.year, format='%Y')
movies1['year'] = pd.DatetimeIndex(movies1['year']).year
movies1.head()

In [ ]:
movies_rating = movies1.groupby('year')['rating'].mean()
movies_rating.head()

In [ ]:
ax = movies_rating.plot.line();
ax.set_title('Average Movie Rating per Year', y=1.04)
ax.set_ylabel('Rating');

Based on the realease movies by years movies that were produced in 1940 till 2020 have the highest averahe rating as compard to 1900 as the year incereases so as the number of ratings however there is a slight decrease from 1980.

In [ ]:
movies_year = movies1.groupby('year')['movieId'].count()

# Plot the movies count per year
ax = movies_year.plot.line();
ax.set_title('Movies count per Year', y=1.04)
ax.set_ylabel('Movies Count')

### Number of ratings per month

In [ ]:
timestamps = df.copy()

movies1['timestamp'] = pd.to_datetime(movies1['timestamp'],unit = 'ms')
movies1['year'] = movies1['timestamp'].dt.year

movies1['month'] = movies1['timestamp'].dt.month

movies1['day'] = movies1['timestamp'].dt.day
movies1['day of week'] = movies1['timestamp'].dt.day_name()
movies1.head()

In [ ]:
movies_year = movies1.groupby('day')['rating'].count()
movies_year

### Number of ratings per day

In [ ]:
fig = plt.figure(figsize=plt.figaspect(.45))
sns.boxplot(y='rating', x='day of week', data=movies1)
plt.show()

In [ ]:
fig, ax = plt.subplots()
sns.countplot(x='day of week', data=movies1, ax=ax)
plt.title('No of ratings on each day...')
plt.ylabel('Total no of ratings')
plt.xlabel('')
ax.set_yticklabels([human(item, 'M') for item in ax.get_yticks()])
plt.show()

## User Analyses

In [ ]:
New_df = Train.join(Movie.set_index('movieId'), on='movieId')
New_df.head()
User = New_df

In [ ]:
from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

data = User['rating'].value_counts().sort_index(ascending=False)
trace = go.Bar(x = data.index,
               text = ['{:.1f} %'.format(val) for val in (data.values / df.shape[0] * 100)],
               textposition = 'auto',
               textfont = dict(color = '#000000'),
               y = data.values,
               )
# Create layout
layout = dict(title = 'Distribution Of {} Movie-ratings'.format(df.shape[0]),
              xaxis = dict(title = 'Rating'),
              yaxis = dict(title = 'Count'))
# Create plot
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

In [ ]:
# Number of ratings per movie
data = User.groupby('movieId')['rating'].count().clip(upper=50)

# Create trace
trace = go.Histogram(x = data.values,
                     name = 'Ratings',
                     xbins = dict(start = 0,
                                  end = 50,
                                  size = 2))
# Create layout
layout = go.Layout(title = 'Distribution Of Number of Ratings Per Movie (Clipped at 100)',
                   xaxis = dict(title = 'Number of Ratings Per Movie'),
                   yaxis = dict(title = 'Count'),
                   bargap = 0.2)

# Create plot
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

In [ ]:
data = User.groupby('userId')['rating'].count().clip(upper=50)

# Create trace
trace = go.Histogram(x = data.values,
                     name = 'Ratings',
                     xbins = dict(start = 0,
                                  end = 50,
                                  size = 2))
# Create layout
layout = go.Layout(title = 'Distribution Of Number of Ratings Per User (Clipped at 50)',
                   xaxis = dict(title = 'Ratings Per User'),
                   yaxis = dict(title = 'Count'),
                   bargap = 0.2)

# Create plot
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

## Genre Analyses

In [ ]:
# defining new DataFrame, to not affect original
genre_rate=movies2.copy()

In [ ]:
# Function to clean text
def clean1(text):
    text =text.replace('(','')
    text =text.replace('|','')
    text =text.replace(')','')
    text =text.replace('-','')
    return text

In [ ]:
genre_rate['genres']=genre_rate['genres'].str.replace('|',",")
genre_rate.head()

In [ ]:
genre_rate['Genre_count'] =genre_rate.genres.str.split(',').str.len()

In [ ]:
genre_rate

In [ ]:
genre_count_stats = genre_rate.groupby('Genre_count')['rating'].mean()


In [ ]:
genre_count_stats = genre_rate.groupby('Genre_count')['rating'].mean()
ax1= genre_count_stats.plot.bar(subplots=True);
plt.title('# average ratings based on number of genres')
plt.xlabel('ratings')
plt.ylabel('No of ratings');

In [ ]:
# Defining
genre_3_df = genre_rate[genre_rate.Genre_count == 7]
genre_3_df = genre_3_df.groupby('genres')['Genre_count', 'rating'].mean()

# Plotting
ax = genre_3_df.rating.sort_values(ascending=False).head(5).plot.barh();
ax.set_xlabel('Rating');

# 4. Clustering
Cluster analysis or clustering is the task of grouping a set of objects in such a way that objects in the same group (called a cluster) are more similar (in some sense) to each other than to those in other groups (clusters). It is a main task of exploratory data mining, and a common technique for statistical data analysis in this section we will do a cluster analyses using KMeans and hierachy to evaluate the behavour of users and combine users based on their similarities.

In [ ]:
# Creating and scaling X variable
X = User[:1000].iloc[:, [1, 2]].values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# Creating copies of the data
train2=train.copy()
movie2=movies.copy()

# Modifying the data
train2.drop(columns='timestamp',axis=1,inplace=True)
df2 = pd.merge(train2 , movie2 , how='outer', on='movieId')
from itertools import chain
df2.dropna(inplace=True)

# function to return list from series of comma-separated strings
def chainer(s):
    return list(chain.from_iterable(s.str.split('|')))

# calculate lengths of splits
lens = df2['genres'].str.split('|').map(len)

In [ ]:
new_df2= pd.DataFrame({'userId': np.repeat(df2['userId'], lens),
                    'movieId': np.repeat(df2['movieId'], lens),
                    'rating': np.repeat(df2['rating'], lens),
                    'genres': chainer(df2['genres'])})                                                
new_df2

In [ ]:
def get_genre_ratings(new_df2, genres, column_names):
    genre_ratings = pd.DataFrame()
    for genre in genres:        
        genre_movies = new_df2[new_df2['genres'].str.contains(genre) ]
        avg_genre_votes_per_user = new_df2[new_df2['movieId'].isin(genre_movies['movieId'])].loc[:, ['userId', 'rating']].groupby(['userId'])['rating'].mean().round(2)
        
        genre_ratings = pd.concat([genre_ratings, avg_genre_votes_per_user], axis=1)
        
    genre_ratings.columns = column_names
    return genre_ratings

## Elbow Method

In [ ]:
# Creating elbow plot
wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', random_state = 42)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)
plt.plot(range(1, 11), wcss)
plt.title('The Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

We use this visualization in order to obtain the optimal clusters for the data, we look for the bend point in the data, as we try to optimize the computational power while still having enough data to accuratly build a model from the data.

In [ ]:
# Clustering with Kmeans
kmeans = KMeans(n_clusters = 3, init = 'k-means++', random_state = 42,max_iter=300)
y_kmeans = kmeans.fit_predict(X)

In [ ]:
plt.scatter(X[y_kmeans == 0, 0], X[y_kmeans == 0, 1], s = 100, c = 'red', label = 'Cluster 1')
plt.scatter(X[y_kmeans == 1, 0], X[y_kmeans == 1, 1], s = 100, c = 'blue', label = 'Cluster 2')
plt.scatter(X[y_kmeans == 2, 0], X[y_kmeans == 2, 1], s = 100, c = 'green', label = 'Cluster 3')
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], s = 300, c = 'yellow', label = 'Centroids')
# plot the centroids
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1],s=250, marker='*',c='red', edgecolor='black',label='centroids')
plt.legend(scatterpoints=1)

plt.show()

## Hierarchy cluster with Dendrogram

In [ ]:
import scipy.cluster.hierarchy as sch
dendrogram = sch.dendrogram(sch.linkage(X, method = 'ward'))
plt.title('Dendrogram')
plt.xlabel('user')
plt.ylabel('no of rating')
plt.show()

This is another way to visualize the data to obtain the optimal amount of clusters similar to the Elbow Method , we use this to pick the clusters that will maximize our efficientcy in the modelling process.

In [ ]:
from sklearn.cluster import AgglomerativeClustering
hc = AgglomerativeClustering(n_clusters = 4, affinity = 'euclidean', linkage = 'ward')
y_hc = hc.fit_predict(X)

In [ ]:
plt.scatter(X[y_hc == 0, 0], X[y_hc == 0, 1], s = 100, c = 'red', label = 'Cluster 1')
plt.scatter(X[y_hc == 1, 0], X[y_hc == 1, 1], s = 100, c = 'blue', label = 'Cluster 2')
plt.scatter(X[y_hc == 2, 0], X[y_hc == 2, 1], s = 100, c = 'green', label = 'Cluster 3')

plt.title('Clusters of user')
plt.xlabel('ratings')
plt.ylabel('movies)')
plt.legend()
plt.show()

# 4. Recommender system (Modeling)

## Content Based Filtering
This type of filter does not involve other users if not ourselves. Based on what we like, the algorithm will simply pick items with similar content to recommend us, based on our personal history with the items that we chose.

<img src= " https://image.ibb.co/f6mDXU/conten.png "  width=45%>

In [ ]:
movie =pd.merge(movies,tags,on='movieId')
movie.drop(columns='timestamp',inplace=True)
movie

In [ ]:
movie['genres']=movie['genres'].str.replace('|','')
movie.fillna('',inplace=True)
movie['metadata']=movie[['genres','tag']].apply(lambda x:''.join(x),axis=1)
movie

In [ ]:
ratings=pd.merge(train,movies,on='movieId')
ratings.drop(columns=['timestamp','genres'],inplace=True)
ratings

In [ ]:
# Entity featurization and similarity computation
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# Libraries used during sorting procedures.
import operator # <-- Convienient item retrieval during iteration 
import heapq # <-- Efficient sorting of large lists

# Imported for our sanity
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df_new = User[:1000]

In [ ]:
df_new 

In [ ]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1,2),
                     min_df=0, stop_words='english')

# Produce a feature matrix, where each row corresponds to a book,
# with TF-IDF features as columns 
tf_authTags_matrix = tf.fit_transform(df_new['title'])

In [ ]:
similar = cosine_similarity(tf_authTags_matrix,
                                        tf_authTags_matrix)
print (similar.shape)

In [ ]:
similar[:5]

In [ ]:
util_matrix = df_new.pivot_table(index=['userId'],
                                       columns=['title'],
                                       values='rating')
util_matrix.shape

In [ ]:
# Create a neat version of the utility matrix to assist with plotting book titles 
df_new['title'] = df_new['title'].apply(lambda x: x[:20])
util_matrix_neat = df_new.pivot_table(index=['userId'],
                                            columns=['title'],
                                            values='rating')

fig, ax = plt.subplots(figsize=(15,5))
# We select only the first 100 users for ease of computation and visualisation. 
# You can play around with this value to see more of the utility matrix. 
_ = sns.heatmap(util_matrix_neat[:100], annot=False, ax=ax).set_title('Movie')

In [ ]:
titles = movie['title']
indices = pd.Series(movie.index, index=movie['title'])

In [ ]:
indices

In [ ]:
ratings_f=train.groupby('userId').filter(lambda x:len(x)<=2)
movie_list_rating =ratings_f.movieId.unique().tolist()

In [ ]:
ratings_f

In [ ]:
movie =movie[movie.movieId.isin(movie_list_rating)]

In [ ]:
movie

In [ ]:
tf = TfidfVectorizer(analyzer = 'word', ngram_range = (1,2),
                     min_df = 0, stop_words = 'english')

# Produce a feature matrix, where each row corresponds to a book,
# with TF-IDF features as columns 
tf_authTags_matrix = tf.fit_transform(movie['metadata']) 

In [ ]:
cosine_sim_authTags = cosine_similarity(tf_authTags_matrix, 
                                        tf_authTags_matrix)
print (cosine_sim_authTags.shape)

In [ ]:
def content_generate_top_N_recommendations(title, N=10):
    # Convert the string book title to a numeric index for our 
    # similarity matrix
    b_idx = indices[title]
    # Extract all similarity values computed with the reference book title
    sim_scores = list(enumerate(cosine_sim_authTags[b_idx]))
    # Sort the values, keeping a copy of the original index of each value
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # Select the top-N values for recommendation
    sim_scores = sim_scores[1:N]
    # Collect indexes 
    book_indices = [i[0] for i in sim_scores]
    # Convert the indexes back into titles 
    return titles.iloc[book_indices]

In [ ]:
# content_generate_top_N_recommendations('Casino (1995)', N=10)

In [ ]:
def content_generate_rating_estimate(title, user, rating_data, k=20, threshold=0.0):
    # Convert the book title to a numeric index for our 
    # similarity matrix
    b_idx = indices[title]
    neighbors = [] # <-- Stores our collection of similarity values 
     
    # Gather the similarity ratings between each book the user has rated
    # and the reference book 
    for index, row in rating_data[rating_data['user_id']==user].iterrows():
        sim = cosine_sim_authTags[b_idx-1, indices[row['title']]-1]
        neighbors.append((sim, row['rating']))
    # Select the top-N values from our collection
    k_neighbors = heapq.nlargest(k, neighbors, key=lambda t: t[0])

    # Compute the weighted average using similarity scoress and 
    # user item ratings. 
    simTotal, weightedSum = 0, 0
    for (simScore, rating) in k_neighbors:
        # Ensure that similarity ratings are above a given threshold
        if (simScore > threshold):
            simTotal += simScore
            weightedSum += simScore * rating
    try:
        predictedRating = weightedSum / simTotal
    except ZeroDivisionError:
        # Cold-start problem - No ratings given by user. 
        # We use the average rating for the reference item as a proxy in this case 
        predictedRating = np.mean(rating_data[rating_data['title']==book_title]['rating'])
    return predictedRating

### Comet

In [ ]:
# params = {'model' = 'tf', 
#          'analyzer' = 'word',
#          'ngram_range' = (1,2),
#          'min_df' = 0}
# experiment.log_params(params)

## Collabartive Based Filtering (App)
This type of filter involves other users. Based on what we like, the algorithm will search for similar users who like the same items and reccomend items to us that they have liked.

![](http://ars.els-cdn.com/content/image/1-s2.0-S0167739X17308890-gr1.jpg )

In [ ]:
# Modifying the Data

tr = train.copy()
tr = tr.drop('timestamp', axis=1) # Removing the unwanted column

merged = pd.merge(tr,movies,on='movieId') # Merging the dataframes
merge = merged.copy()
merge = merged.drop('genres', axis=1) # Dropping the unwanted columns

merge = merge[:500000] # Slicing the data so that there is less computational power required
m = merge.pivot_table(index=['userId'],columns=['title'],values='rating') # pivoting the table into a matrix

userRatings = m.dropna(thresh=10, axis=1).fillna(0,axis=1)
corrMatrix = userRatings.corr(method='pearson') # using pearson correlation
corrMatrix

In [ ]:
# Creating Function to extract similar movies

def get_similar(movie_name,rating=5):
    similar_ratings = corrMatrix[movie_name]*(rating-2.5)
    similar_ratings = similar_ratings.sort_values(ascending=False)
    return similar_ratings

In [ ]:
# Creating a list for 3 Movies, for the recommendations to be based on

fav_movies = [("1984 (1956)",5),("Terminator, The (1984)",5),("Back to the Future Part III (1990)",5)]

similar_movies = pd.DataFrame() # creating empty DataFrame
for movie,rating in fav_movies:
    similar_movies = similar_movies.append(get_similar(movie,rating),ignore_index = True) # Filling up the DataFrame with similar movies

similar_movies.head(10)

In [ ]:
# Calling the function
recc_movies = similar_movies.sum().sort_values(ascending=False).head(40)[3:13]

print('Top recommendations based on your movie choices:\n')
count = 1

for key, value in dict(recc_movies).items():
    print(str(count) + '. ' + str(key))
    count += 1

In [ ]:
from surprise import Reader, Dataset
reader = Reader()
df = train[:50000]
data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)

In [ ]:
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(data, test_size=0.10)

In [ ]:
from surprise import SVD, accuracy
algo = SVD()
algo.fit(trainset)

In [ ]:
predictions = algo.test(testset)

In [ ]:
from surprise import accuracy
accuracy.rmse(predictions)

In [ ]:
pre=[]
for _,row in test.iterrows():
    x_unseen=algo.predict(row['userId'],row['movieId'])
    pred=x_unseen[3]
    pre.append(pred)

# Kaggle Submission

In [ ]:
id = []
for i in test[['userId', 'movieId']].values:
    id.append(str(i[0]) + '_' + str(i[1]))
# new_train['id'] = new_train[]
test['id'] = id

In [ ]:
final_test=test['id']

In [ ]:
Final_table={'id':final_test,'rating':np.round(pre,1)}
submission=pd.DataFrame(data=Final_table)
submission=submission[['id','rating']]

In [ ]:
submission.to_csv('New20.csv',index=False)

### Comet

In [ ]:
# experiment.end()